In [1]:
import cv2 as cv
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

In [3]:
# Define custom metrics and loss functions
def iou_metric(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.round(y_pred), tf.float32)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    return (intersection + smooth) / (union + smooth)


# 1) GPU config & mixed precision (paste near top)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for g in gpus:
        tf.config.experimental.set_memory_growth(g, True)
    try:
        from tensorflow.keras import mixed_precision
        mixed_precision.set_global_policy('mixed_float16')
    except Exception:
        pass

# Load the trained segmentation model
model = tf.keras.models.load_model(
    "Trained_Model - (OriginalPARAM)\\road_segmentation_unet100img.keras",
    custom_objects={
        'iou_metric': iou_metric,
        'bce_dice_loss': bce_dice_loss,
        'dice_loss': dice_loss,
        'dice_coef': dice_coef
    }
)

# after loading model
@tf.function
def model_infer(x):
    # ensure input dtype matches model expectation (float32 if not using mixed)
    return model(x, training=False)

cap = cv.VideoCapture(0)  # 0 = default webcam
if not cap.isOpened():
    raise RuntimeError("Could not open webcam (index 0). Try a different index or check permissions.")



# Smoothing
prev_mask = None
alpha = 0.6  # temporal smoothing strength
kernel = np.ones((5, 5), np.uint8)  # morphological smoothing kernel

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Frame grab failed, exiting.")
            break

        h, w, _ = frame.shape
        # Ensure ROI has height > 0
        y0 = h // 2
        roi = frame[y0:h, :]

        # prepare model input
        resized_frame = cv.resize(roi, (256, 256), interpolation=cv.INTER_LINEAR)
        inp = (resized_frame.astype(np.float32) / 255.0)[None, ...]  # shape (1,256,256,3)
        inp_tf = tf.convert_to_tensor(inp, dtype=tf.float32)         # explicit float32

        # inference (on GPU if available)
        pred_tf = model_infer(inp_tf)
        pred_np = pred_tf.numpy()[0, :, :, 0]  # shape (256,256)

        # binary mask (threshold)
        binary_mask = (pred_np > 0.5).astype(np.uint8)

        # morphological smoothing to clean small holes/noise
        binary_mask = cv.morphologyEx(binary_mask, cv.MORPH_CLOSE, kernel)
        binary_mask = cv.morphologyEx(binary_mask, cv.MORPH_OPEN, kernel)

        # resize back to ROI size and apply temporal smoothing
        mask_resized = cv.resize(binary_mask, (w, h - y0), interpolation=cv.INTER_NEAREST)

        if prev_mask is None:
            smooth_mask = mask_resized.astype(np.float32)
        else:
            smooth_mask = alpha * prev_mask + (1.0 - alpha) * mask_resized
        prev_mask = smooth_mask
        mask_resized = (smooth_mask > 0.5).astype(np.uint8)

        # mask into full frame
        full_mask = np.zeros((h, w), dtype=np.uint8)
        full_mask[y0:h, :] = mask_resized

        # overlay (fast in-place)
        overlay = frame.copy()
        overlay[full_mask == 1] = [0, 255, 0]
        output_frame = cv.addWeighted(frame, 0.7, overlay, 0.3, 0)

        # display
        cv.imshow('Road Segmentation (press q to exit)', output_frame)

        # important: waitKey so window updates and we can break on keypress
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv.destroyAllWindows()